In [1]:
import os
import gc
import glob
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from rasterio.enums import Resampling
import colorcet  # Extra colormaps
import numpy as np
import rasterio

# These functions could be used independetly

In [70]:
import matplotlib.pyplot as plt
import colorcet  # Extra colormaps

# Define available colormaps
COLORMAPS = {
    # 🌎 Extra Colorcet Colormaps (for better color mapping)
    "rainbow4": colorcet.cm.rainbow4,  # Improved rainbow
}

DEFAULT_COLORMAP = "rainbow4"
NDVI_MIN, NDVI_MAX = 0.5, 1.0  # NDVI range

In [3]:
def find_orthomosaic_files(input_folder, search_config, DEFAULT_COLORMAP=DEFAULT_COLORMAP):
    """
    Find orthomosaic files based on the path structure and search configuration.
    
    Args:
        input_folder (str): The base directory where field_name/key_word/ files are located.
        search_config (dict): Dictionary with keywords and suffixes to search for.
        
    Returns:
        dict: Dictionary with keyword as keys and field names with their file lists.
    """
    orthomosaic_files = {}

    # Walk through the field_name folders inside input_folder
    for field_name in os.listdir(input_folder):
        field_path = os.path.join(input_folder, field_name)
        
        # Skip if not a directory
        if not os.path.isdir(field_path):
            continue
        
        for key_word, (suffix, _) in search_config.items():
            # Search for files under the key_word folder
            key_word_folder = os.path.join(field_path, key_word, DEFAULT_COLORMAP)
            # If the key_word folder exists, search for orthomosaics
            if os.path.isdir(key_word_folder):
                file_paths = glob.glob(os.path.join(key_word_folder, f'*{suffix}'))
                if file_paths:
                    if key_word not in orthomosaic_files:
                        orthomosaic_files[key_word] = {}
                    orthomosaic_files[key_word][field_name] = file_paths

    return orthomosaic_files


In [4]:
import rasterio
import numpy as np
from PIL import Image

def rotate_rgb_tif(file_path, angle, save=False, output_path=None):
    """
    Rotate an RGB TIFF image by a given angle.
    
    Args:
        file_path (str): Path to the input RGB TIFF image.
        angle (float): The angle by which to rotate the image (in degrees).
        save (bool): Flag to decide whether to save the rotated image. Default is False.
        output_path (str): Path to save the rotated image (only used if save=True).
    
    Returns:
        rotated_image (PIL.Image): The rotated image (PIL format).
    """
    # Open the TIFF file using rasterio
    with rasterio.open(file_path) as src:
        # Read the three RGB bands
        red = src.read(1)  # First band: Red
        green = src.read(2)  # Second band: Green
        blue = src.read(3)  # Third band: Blue

    # Stack the bands to create a 3D RGB image
    rgb_image = np.dstack((red, green, blue))

    # Convert the NumPy array to a PIL Image
    pil_image = Image.fromarray(rgb_image)

    # Rotate the image
    rotated_image = pil_image.rotate(angle, resample=Image.BICUBIC, expand=True)

    # If save flag is True, save the rotated image
    if save:
        if not output_path:
            raise ValueError("Output path must be specified if save=True.")
        
        # Convert the rotated PIL image back to a NumPy array
        rotated_rgb = np.array(rotated_image)

        # Write the rotated image back to a TIFF file
        with rasterio.open(output_path, 'w', driver='GTiff', 
                           count=3, width=rotated_rgb.shape[1], height=rotated_rgb.shape[0], 
                           dtype=rotated_rgb.dtype) as dst:
            # Write the rotated bands to the new file
            dst.write(rotated_rgb[:, :, 0], 1)  # Red channel
            dst.write(rotated_rgb[:, :, 1], 2)  # Green channel
            dst.write(rotated_rgb[:, :, 2], 3)  # Blue channel

        print(f"✅ Image rotated and saved to {output_path}")

    # Return the rotated image without saving (if save=False)
    return rotated_image

In [ ]:
import numpy as np
import rasterio

def crop_ndvi_image(image_path, output_path, crop_values=(0, 108, 3)):
    """Crop an NDVI image by removing rows and columns that contain only 0, 108 and 3."""
    with rasterio.open(image_path) as src:
        rgb = src.read().transpose(1, 2, 0)  # Convert (Bands, Height, Width) → (Height, Width, Bands)

        # Identify valid columns (contain at least one pixel not in crop_values)
        valid_cols = np.any(~np.isin(rgb, crop_values), axis=(0, 2))
        left = np.argmax(valid_cols)  # First valid column
        right = len(valid_cols) - np.argmax(valid_cols[::-1])  # Last valid column

        # Identify valid rows (contain at least one pixel not in crop_values)
        valid_rows = np.any(~np.isin(rgb, crop_values), axis=(1, 2))
        top = np.argmax(valid_rows)  # First valid row
        bottom = len(valid_rows) - np.argmax(valid_rows[::-1])  # Last valid row

        # Crop the image
        cropped_rgb = rgb[top:bottom, left:right, :]

        # Update profile and save
        profile = src.profile
        profile.update(height=cropped_rgb.shape[0], width=cropped_rgb.shape[1])

        with rasterio.open(output_path, "w", **profile) as dst:
            for i in range(3):  # Save each RGB band
                dst.write(cropped_rgb[:, :, i], i + 1)

    print(f"✅ Cropped image saved to: {output_path}")
    # Free memory
    del rgb
    gc.collect()


# The following performs rotation only using above function

In [ ]:
def process_rotation_and_save(orthomosaic_files, rotation_dict, output_folder):
    """
    Process each orthomosaic based on the rotation_dict, rotate images, and save them.
    
    Args:
        orthomosaic_files (dict): Dictionary of files to be processed.
        rotation_dict (dict): Dictionary of rotation angles per field_name.
        output_folder (str): The output directory where rotated images will be saved.
    """
    for key_word, field_dict in orthomosaic_files.items():
        for field_name, file_list in field_dict.items():
            # Check if there is a predefined rotation for the field_name
            if field_name in rotation_dict:
                rotation_angle = rotation_dict[field_name]
                print(f"Rotating {field_name} with {rotation_angle}°...")

                for file_path in file_list:
                    # Define the output path for the rotated image
                    file_name = os.path.basename(file_path)
                    # output_path = os.path.join(output_folder, field_name, key_word, file_name)
                    
                    # Identify file type (TIFF/JPG) based on extension
                    file_ext = os.path.splitext(file_path)[-1].lower()
                    output_path = os.path.join(os.path.dirname(file_path)+"_rotated", 
                                               os.path.basename(file_path).replace(file_ext, f"_{"rotated"}%{file_ext}"))

                    # Ensure the output folder exists
                    os.makedirs(os.path.dirname(output_path), exist_ok=True)

                    # Apply rotation & save the rotated image
                    rotated_image =  rotate_rgb_tif(file_path, rotation_angle, save=True, output_path=output_path)
                    print(f"✅ Saved rotated image: {output_path}")
            else:
                print(f"⚠️ No rotation defined for {field_name}. Skipping rotation.")

In [ ]:
# Usage:

# Define the input folder where the projects and orthomosaics are stored
input_folder = r"D:\PhenoCrop\NDVI_rainbow4_0.5-1.0"
# Define the output folder to save colorized files
output_folder = input_folder

# Example search configuration to find TIFF or JPG files
search_config = {
    "NDVI": ("_index_ndvi.tif", _), # TIFF files scaled to 50%
}

# Rotation angles for specific fields (not used directly here but could be added in the future)
rotation_dict = {
     'PHENO_CROP': -13.5,
     'OAT_FRONTIERS': -13.5,
     'DIVERSITY_OATS': -13.5,
     'PRO_BAR_VOLL': -29.5,
     'E166': -115.5,
     'PRO_BAR_SØRÅS': -68.5
}

# Find orthomosaic files
orthomosaic_files = find_orthomosaic_files(input_folder, search_config)
# orthomosaic_files

In [ ]:
# process_rotation_and_save(orthomosaic_files, rotation_dict, output_folder)

# Unified use of rotation and cropping

# All in one working solution for rotation and cropping

In [63]:
import os
import numpy as np
import rasterio
from PIL import Image

def rotate_rgb_tif(file_path, angle, save=False, output_path=None):
    """
    Rotate an RGB TIFF image by a given angle.
    
    Args:
        file_path (str): Path to the input RGB TIFF image.
        angle (float): The angle by which to rotate the image (in degrees).
        save (bool): Flag to decide whether to save the rotated image. Default is False.
        output_path (str): Path to save the rotated image (only used if save=True).
    
    Returns:
        rotated_image (PIL.Image): The rotated image (PIL format).
    """
    # Open the TIFF file using rasterio
    with rasterio.open(file_path) as src:
        # Read the three RGB bands
        red = src.read(1)  # First band: Red
        green = src.read(2)  # Second band: Green
        blue = src.read(3)  # Third band: Blue

    # Stack the bands to create a 3D RGB image
    rgb_image = np.dstack((red, green, blue))

    # Convert the NumPy array to a PIL Image
    pil_image = Image.fromarray(rgb_image)

    # Rotate the image
    rotated_image = pil_image.rotate(angle, resample=Image.BICUBIC, expand=True)

    # If save flag is True, save the rotated image
    if save:
        if not output_path:
            raise ValueError("Output path must be specified if save=True.")
        
        # Convert the rotated PIL image back to a NumPy array
        rotated_rgb = np.array(rotated_image)

        # Write the rotated image back to a TIFF file
        with rasterio.open(output_path, 'w', driver='GTiff', 
                           count=3, width=rotated_rgb.shape[1], height=rotated_rgb.shape[0], 
                           dtype=rotated_rgb.dtype) as dst:
            # Write the rotated bands to the new file
            dst.write(rotated_rgb[:, :, 0], 1)  # Red channel
            dst.write(rotated_rgb[:, :, 1], 2)  # Green channel
            dst.write(rotated_rgb[:, :, 2], 3)  # Blue channel

        print(f"✅ Image rotated and saved to {output_path}")

    # Return the rotated image without saving (if save=False)
    return rotated_image

def crop_ndvi_image(rotated_image, output_path, crop_values=(0, 108, 3)):
    """
    Crop an NDVI image by removing rows and columns that contain only specific values.
    
    Args:
        rotated_image (PIL.Image): The rotated NDVI image (PIL format).
        output_path (str): Path to save the cropped image.
        crop_values (tuple): Pixel values that define empty areas.
    """
    try:
        # Convert PIL Image to NumPy array
        rotated_rgb = np.array(rotated_image)

        # Identify valid columns
        valid_cols = np.any(~np.isin(rotated_rgb, crop_values), axis=(0, 2))
        left, right = np.argmax(valid_cols), len(valid_cols) - np.argmax(valid_cols[::-1])

        # Identify valid rows
        valid_rows = np.any(~np.isin(rotated_rgb, crop_values), axis=(1, 2))
        top, bottom = np.argmax(valid_rows), len(valid_rows) - np.argmax(valid_rows[::-1])

        # Crop image
        cropped_rgb = rotated_rgb[top:bottom, left:right, :]

        # Save cropped image
        with rasterio.open(output_path, 'w', driver='GTiff', 
                           count=3, width=cropped_rgb.shape[1], height=cropped_rgb.shape[0], 
                           dtype=cropped_rgb.dtype) as dst:
            # Write the cropped bands to the new file
            dst.write(cropped_rgb[:, :, 0], 1)  # Red channel
            dst.write(cropped_rgb[:, :, 1], 2)  # Green channel
            dst.write(cropped_rgb[:, :, 2], 3)  # Blue channel

        print(f"✅ Cropped image saved: {output_path}")

    except Exception as e:
        print(f"❌ ERROR cropping image: {str(e)}")

def process_rotation_and_crop_and_save(orthomosaic_files, rotation_dict):
    """
    Process each orthomosaic, rotate & crop images, and save them.

    Args:
        orthomosaic_files (dict): Dictionary of files to process.
        rotation_dict (dict): Rotation angles per field_name.
    """
    for key_word, field_dict in orthomosaic_files.items():
        for field_name, file_list in field_dict.items():
            if field_name in rotation_dict:
                rotation_angle = rotation_dict[field_name]
                print(f"🔄 Processing {field_name} | Rotation: {rotation_angle}°")

                for file_path in file_list:
                    file_name = os.path.basename(file_path)
                    file_ext = os.path.splitext(file_path)[-1].lower()

                    # Create output path
                    output_path = os.path.join(os.path.dirname(file_path)+"_rotated", 
                           os.path.basename(file_path).replace(file_ext, f"_{"rotated"}{file_ext}"))

                    os.makedirs(os.path.dirname(output_path), exist_ok=True)

                    # 🚀 Rotate Image
                    rotated_image = rotate_rgb_tif(file_path, rotation_angle, save=False)

                    # ✂️ Crop and Save Image
                    crop_ndvi_image(rotated_image, output_path)

                    print(f"✅ Processed image saved: {output_path}")
            else:
                print(f"⚠️ No rotation defined for {field_name}. Skipping.")

In [64]:
def find_orthomosaic_files_colored_ndvi(input_folder, search_config, DEFAULT_COLORMAP=DEFAULT_COLORMAP):
    """
    Find orthomosaic files based on the path structure and search configuration.
    
    Args:
        input_folder (str): The base directory where field_name/key_word/ files are located.
        search_config (dict): Dictionary with keywords and suffixes to search for.
        
    Returns:
        dict: Dictionary with keyword as keys and field names with their file lists.
    """
    orthomosaic_files = {}

    # Walk through the field_name folders inside input_folder
    for field_name in os.listdir(input_folder):
        field_path = os.path.join(input_folder, field_name)
        
        # Skip if not a directory
        if not os.path.isdir(field_path):
            continue
        
        for key_word, (suffix, _) in search_config.items():
            # Search for files under the key_word folder
            key_word_folder = os.path.join(field_path, key_word, DEFAULT_COLORMAP)
            # If the key_word folder exists, search for orthomosaics
            if os.path.isdir(key_word_folder):
                file_paths = glob.glob(os.path.join(key_word_folder, f'*{suffix}'))
                if file_paths:
                    if key_word not in orthomosaic_files:
                        orthomosaic_files[key_word] = {}
                    orthomosaic_files[key_word][field_name] = file_paths

    return orthomosaic_files


In [65]:
def find_orthomosaic_files_rgb(input_folder, search_config):
    """
    Find orthomosaic files based on the path structure and search configuration.
    
    Args:
        input_folder (str): The base directory where field_name/key_word/ files are located.
        search_config (dict): Dictionary with keywords and suffixes to search for.
        
    Returns:
        dict: Dictionary with keyword as keys and field names with their file lists.
    """
    orthomosaic_files = {}

    # Walk through the field_name folders inside input_folder
    for field_name in os.listdir(input_folder):
        field_path = os.path.join(input_folder, field_name)
        
        # Skip if not a directory
        if not os.path.isdir(field_path):
            continue
        
        for key_word, (suffix, _) in search_config.items():
            # Search for files under the key_word folder
            key_word_folder = os.path.join(field_path, key_word)
            # If the key_word folder exists, search for orthomosaics
            if os.path.isdir(key_word_folder):
                file_paths = glob.glob(os.path.join(key_word_folder, f'*{suffix}'))
                if file_paths:
                    if key_word not in orthomosaic_files:
                        orthomosaic_files[key_word] = {}
                    orthomosaic_files[key_word][field_name] = file_paths

    return orthomosaic_files


In [66]:
# Usage:

# Define the input folder where the projects and orthomosaics are stored
input_folder = r"D:\PhenoCrop\3_orthomosaics_rgb_ndvi"
# Define the output folder to save colorized files
output_folder = r"D:\PhenoCrop\NDVI_rainbow4_0.5-1.0_2"

# Example search configuration to find TIFF or JPG files
search_config = {
    "RGB": ("_transparent_mosaic_group1.tif", 0.25), # JPG files scaled to 25%
    # "NDVI": ("_index_ndvi.tif", _), # TIFF files scaled to 50%
}

# Rotation angles for specific fields (not used directly here but could be added in the future)
rotation_dict = {
     'PHENO_CROP': -13.5,
     'OAT_FRONTIERS': -13.5,
     'DIVERSITY_OATS': -13.5,
     'PRO_BAR_VOLL': -29.5,
     'E166': -115.5,
     'PRO_BAR_SØRÅS': -68.5
}

# Find orthomosaic files
orthomosaic_files_colored_ndvi = find_orthomosaic_files_colored_ndvi(input_folder, search_config)
orthomosaic_files_rgb = find_orthomosaic_files_rgb(input_folder, search_config)
# orthomosaic_files_rgb

In [47]:
def delete_inner_keys(nested_dict, field_name, key_word):
    """Deletes a specific inner key (key_word) from a nested dictionary (field_name level)."""
    if field_name in nested_dict and key_word in nested_dict[field_name]:
        del nested_dict[field_name][key_word]
    #     print(f"Deleted key '{key_word}' from '{field_name}'")
    # else:
    #     print(f"Key '{key_word}' not found in '{field_name}'")

# delete_fields = [ "PILOT", "PHENO_CROP", "OAT_FRONTIERS", "E166", "DIVERSITY_OATS", "PRO_BAR_VOLL"]
delete_fields = ["PRO_BAR_VOLL", "PHENO_CROP", "OAT_FRONTIERS", "E166", "DIVERSITY_OATS"]

for field in delete_fields:
    delete_inner_keys(orthomosaic_files, "NDVI", field)

In [68]:
process_rotation_and_crop_and_save(orthomosaic_files_rgb, rotation_dict)

🔄 Processing DIVERSITY_OATS | Rotation: -13.5°
✅ Cropped image saved: D:\PhenoCrop\3_orthomosaics_rgb_ndvi\DIVERSITY_OATS\RGB_rotated\20240603 DIVERSITY_OATS M3M 30m MS 80 85_transparent_mosaic_group1_25%_rotated.tif
✅ Processed image saved: D:\PhenoCrop\3_orthomosaics_rgb_ndvi\DIVERSITY_OATS\RGB_rotated\20240603 DIVERSITY_OATS M3M 30m MS 80 85_transparent_mosaic_group1_25%_rotated.tif
✅ Cropped image saved: D:\PhenoCrop\3_orthomosaics_rgb_ndvi\DIVERSITY_OATS\RGB_rotated\20240607 DIVERSITY_OATS M3M 30m MS 80 85_transparent_mosaic_group1_25%_rotated.tif
✅ Processed image saved: D:\PhenoCrop\3_orthomosaics_rgb_ndvi\DIVERSITY_OATS\RGB_rotated\20240607 DIVERSITY_OATS M3M 30m MS 80 85_transparent_mosaic_group1_25%_rotated.tif
✅ Cropped image saved: D:\PhenoCrop\3_orthomosaics_rgb_ndvi\DIVERSITY_OATS\RGB_rotated\20240611 DIVERSITY_OATS M3M 30m MS 80 85_transparent_mosaic_group1_25%_rotated.tif
✅ Processed image saved: D:\PhenoCrop\3_orthomosaics_rgb_ndvi\DIVERSITY_OATS\RGB_rotated\20240611 D

# Add a colorbar to the colored NDVI Ortho

# Colorbar integration, good

# Make the ortho background transparent

In [9]:
import numpy as np
import rasterio
from matplotlib import pyplot as plt
from PIL import Image
import colorcet  # Import the colorcet library

def add_ndvi_colorbar(orthomosaic_path, output_path, ndvi_palette, ndvi_range, colorbar_height_ratio=0.5, colorbar_width_ratio=0.05, background_value=(0, 0, 0), ortho_background="transparent", colorbar_background="transparent", colorbar_margin_background="transparent"):
    """
    Add an NDVI color bar to the right side of an orthomosaic. The color bar is generated in its original size and then scaled.
    The background of the input orthomosaic, color bar, and the margin around the color bar can be customized.

    Args:
        orthomosaic_path (str): Path to the input orthomosaic.
        output_path (str): Path to save the output image with the color bar.
        ndvi_palette (matplotlib.colors.Colormap): Colormap for the NDVI color bar (e.g., colorcet.cm.rainbow4).
        ndvi_range (tuple): Range of NDVI values (min, max).
        colorbar_height_ratio (float): Height of the color bar as a ratio of the image height (default: 0.5).
        colorbar_width_ratio (float): Width of the color bar as a ratio of the image width (default: 0.05).
        background_value (tuple): RGB value of the background to make transparent (default: (0, 0, 0)).
        ortho_background (str): Background option for the orthomosaic ("transparent", "white", or "black").
        colorbar_background (str): Background option for the color bar ("transparent", "white", or "black").
        colorbar_margin_background (str): Background option for the margin around the color bar ("transparent", "white", or "black").
    """
    # Load the orthomosaic
    with rasterio.open(orthomosaic_path) as src:
        orthomosaic = src.read()  # Read all bands
        orthomosaic = np.moveaxis(orthomosaic, 0, -1)  # Convert to (Height, Width, Bands)
        height, width = orthomosaic.shape[:2]  # Get image dimensions

    # Convert orthomosaic to RGBA mode
    orthomosaic_pil = Image.fromarray(orthomosaic.astype(np.uint8)).convert("RGBA")
    orthomosaic_data = np.array(orthomosaic_pil)  # Convert to NumPy array for manipulation

    # Handle orthomosaic background
    if ortho_background == "transparent":
        # Identify background pixels (where RGB matches the background_value)
        background_pixels = np.all(orthomosaic_data[:, :, :3] == background_value, axis=-1)
        # Set alpha channel to 0 for background pixels
        orthomosaic_data[background_pixels, 3] = 0  # Set alpha to 0 (fully transparent)
    elif ortho_background == "white":
        # Replace background with white
        background_pixels = np.all(orthomosaic_data[:, :, :3] == background_value, axis=-1)
        orthomosaic_data[background_pixels] = [255, 255, 255, 255]  # White with full opacity
    elif ortho_background == "black":
        # Replace background with black
        background_pixels = np.all(orthomosaic_data[:, :, :3] == background_value, axis=-1)
        orthomosaic_data[background_pixels] = [0, 0, 0, 255]  # Black with full opacity

    # Convert back to PIL Image
    orthomosaic_pil = Image.fromarray(orthomosaic_data)

    # Calculate color bar dimensions
    colorbar_width = int(width * colorbar_width_ratio)  # Width of the color bar
    colorbar_height = int(height * colorbar_height_ratio)  # Height of the color bar

    # Create the NDVI color bar in its original size (higher DPI for better quality)
    fig, ax = plt.subplots(figsize=(0.65, 6))  # Original size of the color bar
    fig.subplots_adjust(right=0.5)

    # Use the provided colormap
    cmap = ndvi_palette
    norm = plt.Normalize(vmin=ndvi_range[0], vmax=ndvi_range[1])

    # Add the color bar
    cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), cax=ax, orientation='vertical')
    
    # Customize the color bar (match the style you shared)
    cbar.set_label("NDVI Value", fontsize=12, weight='normal')  # Fixed text size
    cbar.ax.tick_params(labelsize=10)  # Fixed tick label size
    cbar.outline.set_edgecolor('black')  # Add a border to the color bar
    cbar.outline.set_linewidth(1)  # Border thickness

    # Handle color bar background
    if colorbar_background == "transparent":
        # Save the color bar with a transparent background
        colorbar_path = "colorbar_temp.png"
        plt.savefig(colorbar_path, bbox_inches="tight", pad_inches=0.1, dpi=300, transparent=True)
    elif colorbar_background == "white":
        # Save the color bar with a white background
        colorbar_path = "colorbar_temp.png"
        plt.savefig(colorbar_path, bbox_inches="tight", pad_inches=0.1, dpi=300, facecolor="white")
    elif colorbar_background == "black":
        # Save the color bar with a black background
        colorbar_path = "colorbar_temp.png"
        plt.savefig(colorbar_path, bbox_inches="tight", pad_inches=0.1, dpi=300, facecolor="black")

    plt.close()

    # Load the color bar image
    colorbar = Image.open(colorbar_path)

    # Resize the color bar using high-quality resampling (LANCZOS)
    colorbar = colorbar.resize((colorbar_width, colorbar_height), Image.LANCZOS)

    # Create a combined image with the specified margin background
    combined_width = width + colorbar_width
    combined_image = Image.new("RGBA", (combined_width, height), (0, 0, 0, 0))  # Start with transparent background

    # Handle margin background
    if colorbar_margin_background == "white":
        combined_image = Image.new("RGBA", (combined_width, height), (255, 255, 255, 255))  # White background
    elif colorbar_margin_background == "black":
        combined_image = Image.new("RGBA", (combined_width, height), (0, 0, 0, 255))  # Black background

    # Paste the orthomosaic and color bar onto the combined image
    combined_image.paste(orthomosaic_pil, (0, 0))
    combined_image.paste(colorbar, (width, (height - colorbar_height) // 2), colorbar)  # Use colorbar as mask for transparency

    # Save the final image with transparency
    combined_image.save(output_path, format="TIFF", transparency=0)  # Save as TIFF with transparency
    print(f"✅ Image with NDVI color bar saved to {output_path}")

    # Clean up temporary color bar file
    import os
    os.remove(colorbar_path)

In [38]:
# Example usage
file_path = r'D:\PhenoCrop\3_orthomosaics_rgb_ndvi\NDVI_rainbow4_rotated\20240701 PRO_BAR_SØRÅS M3M 30m MS 80 85_index_ndvi_50%_rainbow4_processed.tif'
output_path = r'D:\PhenoCrop\3_orthomosaics_rgb_ndvi\NDVI_rainbow4_rotated\NDVI.tif'
ndvi_palette = colorcet.cm.rainbow4  # Use colorcet.cm.rainbow4 as the NDVI palette
ndvi_range = (0.5, 1)  # NDVI value range
colorbar_height_ratio = 0.48  # Color bar height as 50% of the image height
colorbar_width_ratio = 0.14  # Color bar width as 5% of the image width
background_value = (0, 0, 0)  # RGB value of the background to make transparent (black in this case)


In [39]:
# Background options: "transparent", "white", or "black"
ortho_background = "white"  # Make orthomosaic background transparent
colorbar_background = "white"  # Make color bar background white
colorbar_margin_background = "white"  # Make margin background black

add_ndvi_colorbar(file_path, output_path,
                  ndvi_palette, ndvi_range,
                  colorbar_height_ratio, colorbar_width_ratio,
                  background_value,
                  ortho_background, colorbar_background, colorbar_margin_background)

✅ Image with NDVI color bar saved to D:\PhenoCrop\3_orthomosaics_rgb_ndvi\NDVI_rainbow4_rotated\NDVI.tif


# Apply the colorbar to all the rotated Orthos

In [41]:
import os
import glob
def find_orthomosaic_files_rgb(input_folder, search_config):
    """
    Find orthomosaic files based on the path structure and search configuration.
    
    Args:
        input_folder (str): The base directory where field_name/key_word/ files are located.
        search_config (dict): Dictionary with keywords and suffixes to search for.
        
    Returns:
        dict: Dictionary with keyword as keys and field names with their file lists.
    """
    orthomosaic_files = {}

    # Walk through the field_name folders inside input_folder
    for field_name in os.listdir(input_folder):
        field_path = os.path.join(input_folder, field_name)
        
        # Skip if not a directory
        if not os.path.isdir(field_path):
            continue
        
        for key_word, (suffix, _) in search_config.items():
            # Search for files under the key_word folder
            key_word_folder = os.path.join(field_path, key_word)
            # If the key_word folder exists, search for orthomosaics
            if os.path.isdir(key_word_folder):
                file_paths = glob.glob(os.path.join(key_word_folder, f'*{suffix}'))
                if file_paths:
                    if key_word not in orthomosaic_files:
                        orthomosaic_files[key_word] = {}
                    orthomosaic_files[key_word][field_name] = file_paths

    return orthomosaic_files


In [42]:
import os

def add_suffix_to_file_name(file_path, suffix):
    """
    Adds a suffix to the file name in the given file path.
    
    Parameters:
    - file_path (str): The original file path.
    - suffix (str): The suffix to add to the file name (before the file extension).
    
    Returns:
    - str: The updated file path with the suffix added to the file name.
    """
    # Separate the directory, file name, and extension
    directory, file_name = os.path.split(file_path)
    base_name, ext = os.path.splitext(file_name)
    
    # Add the suffix to the file name
    new_file_name = f"{base_name}{suffix}{ext}"
    
    # Reconstruct the full file path
    new_file_path = os.path.join(directory, new_file_name)
    return new_file_path

In [44]:
input_folder = r'D:\PhenoCrop\3_orthomosaics_rgb_ndvi'
# Example search configuration to find TIFF or JPG files
search_config = {
    "NDVI_rainbow4_rotated": ("_rainbow4_rotated.tif", 0.25), # JPG files scaled to 25%
}

In [45]:
orthomosaic_files_for_colorbar_rainbow4 = find_orthomosaic_files_rgb(input_folder, search_config)

In [47]:
for key_word, field_dict in orthomosaic_files_for_colorbar_rainbow4.items():
    for field_name, file_list in field_dict.items():
        for rainbow4 in file_list:
            add_ndvi_colorbar(rainbow4, add_suffix_to_file_name(rainbow4, "_with_colorbar_HQ"),
                              ndvi_palette, ndvi_range, 
                              colorbar_height_ratio, colorbar_width_ratio,
                              background_value, 
                              ortho_background, colorbar_background, colorbar_margin_background)

✅ Image with NDVI color bar saved to D:\PhenoCrop\3_orthomosaics_rgb_ndvi\DIVERSITY_OATS\NDVI_rainbow4_rotated\20240603 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%_rainbow4_rotated_with_colorbar_HQ.tif
✅ Image with NDVI color bar saved to D:\PhenoCrop\3_orthomosaics_rgb_ndvi\DIVERSITY_OATS\NDVI_rainbow4_rotated\20240607 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%_rainbow4_rotated_with_colorbar_HQ.tif
✅ Image with NDVI color bar saved to D:\PhenoCrop\3_orthomosaics_rgb_ndvi\DIVERSITY_OATS\NDVI_rainbow4_rotated\20240611 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%_rainbow4_rotated_with_colorbar_HQ.tif
✅ Image with NDVI color bar saved to D:\PhenoCrop\3_orthomosaics_rgb_ndvi\DIVERSITY_OATS\NDVI_rainbow4_rotated\20240619 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%_rainbow4_rotated_with_colorbar_HQ.tif
✅ Image with NDVI color bar saved to D:\PhenoCrop\3_orthomosaics_rgb_ndvi\DIVERSITY_OATS\NDVI_rainbow4_rotated\20240624 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%_rainbow4_rota